In [78]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier


In [79]:
filename = 'C:/Users/yuwei/Downloads/lime-master/loan_dataset/selected.csv'
#test_filename = 'C:/Users/yuwei/Downloads/lime-master/loan_dataset/av-sigmacab-test.csv'
dataset = pd.read_csv(filename, header=0)
cols=dataset.columns
print (dataset.shape)
print(cols)

(42538, 44)
Index([u'loan_amnt', u'funded_amnt', u'term', u'int_rate', u'installment',
       u'grade', u'emp_length', u'home_ownership', u'annual_inc',
       u'verification_status', u'loan_status', u'purpose', u'title',
       u'zip_code', u'dti', u'delinq_2yrs', u'earliest_cr_line',
       u'inq_last_6mths', u'mths_since_last_delinq', u'mths_since_last_record',
       u'open_acc', u'pub_rec', u'revol_bal', u'revol_util', u'total_acc',
       u'total_pymnt', u'total_rec_prncp', u'total_rec_int',
       u'total_rec_late_fee', u'recoveries', u'collection_recovery_fee',
       u'last_pymnt_d', u'last_pymnt_amnt', u'last_credit_pull_d',
       u'collections_12_mths_ex_med', u'application_type', u'acc_now_delinq',
       u'chargeoff_within_12_mths', u'delinq_amnt', u'pub_rec_bankruptcies',
       u'tax_liens', u'hardship_flag', u'disbursement_method',
       u'debt_settlement_flag'],
      dtype='object')


In [80]:
target = np.ravel(np.array(dataset['grade'].values))
dataset = dataset.drop(['grade'],axis=1)

In [81]:
feature_names = list(dataset.columns)
print(feature_names)

['loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'loan_status', 'purpose', 'title', 'zip_code', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag']


In [82]:
categorical_columns = []
n = 0
for i in dataset.columns:
    if dataset[i].dtype == 'object':
        categorical_columns.append(n)
    n = n+1
        
print(categorical_columns)

[2, 5, 6, 8, 9, 10, 11, 12, 14, 15, 21, 22, 23, 29, 30, 32, 34, 40, 41, 42]


In [83]:
categorical_names = {}
for feature in categorical_columns:
    #print(feature)
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(dataset.iloc[:,feature])
    dataset.iloc[:,feature] = le.transform(dataset.iloc[:,feature])
    categorical_names[feature] = le.classes_
    #print(le.classes_)

In [84]:
dataset = dataset.astype(float)

In [85]:
imp = Imputer(missing_values='NaN',strategy='most_frequent',axis=0,verbose=0,copy=True)
dataset = imp.fit_transform(dataset) #dataframe to array ?

D:\Program Files (x86)\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [86]:
trainset = dataset[:10000]
train_target = target[:10000]
testset = dataset[10000:]
test_target = target[10000:]
print(trainset.shape)
print(testset.shape)

(10000L, 43L)
(32538L, 43L)


In [87]:
#encoder = sklearn.preprocessing.OneHotEncoder(categorical_features =  categorical_columns)
#encoder.fit(dataset)
#encoded_train = encoder.transform(trainset)  #more scientific but useless (the accuracy will decrease)

In [88]:
model_dt = DecisionTreeClassifier(max_features = 10)
model_dt.fit(trainset, train_target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [89]:
predict_result_dt = model_dt.predict(testset)
print("Accuracy: %f" % accuracy_score(test_target,predict_result_dt))

Accuracy: 0.686182


In [ ]:
from IPython.display import Image  
from sklearn import tree
import pydotplus 
dot_data = tree.export_graphviz(model_dt, out_file=None, 
                         feature_names=feature_names,  
                         class_names = ['A','B','C','D','E','F','G'],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  

In [ ]:
#graph.write_pdf("lendingclub.pdf")

In [ ]:
#Image(graph.create_png())

In [ ]:
print(model_dt.feature_importances_)

In [ ]:
#model_rf = RandomForestClassifier(n_estimators=500)
#model_rf.fit(trainset, train_target)

In [ ]:
#predict_result = model_rf.predict(testset)
#print("Accuracy: %f" % accuracy_score(test_target,predict_result))


LIME

In [ ]:
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
from __future__ import print_function

In [ ]:
#predict_fn_rf = lambda x: model_rf.predict_proba(x).astype(float)

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(trainset ,
                                                   feature_names = feature_names,class_names=['A','B','C','D','E','F','G'],
                                                   categorical_features=categorical_columns, 
                                                   categorical_names=categorical_names, kernel_width=3,verbose = True
                                                 )

In [ ]:
idx = 7645;
#print(testset[idx])
print(test_target[idx])
print(predict_result_dt[idx])
exp = explainer.explain_instance(testset[idx],model_dt.predict_proba, 
                                 num_features=6, labels = [0,1,2,3,4,5,6],
                                 distance_metric = 'cosine') #euclidean
exp.show_in_notebook(show_all=False)


In [ ]:
importances = model_dt.feature_importances_
t1 = [(i,importances[i]) for i in range(len(importances))]
def by_importance(t):
    return -t[1]
t2 = sorted(t1,key = by_importance)
print(t2)

In [ ]:
goldset = set([3,26,22,4,30,17,13,19,24,2])
dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6}
true_num = 0
for i in range(1): #set the number of test set
    topset = set()
    predictlabel = predict_result_dt[i]
    labelnum = dict[predictlabel]
    exp = explainer.explain_instance(testset[i],model_dt.predict_proba,
                                     num_features=3,labels = [labelnum],
                                     distance_metric = 'euclidean')
    for j in range(3):
        topset.add(exp.as_map()[labelnum][j][0]) 
    if topset < goldset:
        true_num = true_num + 1
print(true_num)